In [6]:
import numpy as np
import pandas as pd

# Zbiór danych apartments

In [2]:
df = pd.read_csv("apartments.csv", index_col=False)
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [3]:
df.head()

,m2.price,construction.year,surface,floor,no.rooms,district
0,5897,1953,25,3,1,Srodmiescie
1,1818,1992,143,9,5,Bielany
2,3643,1937,56,1,2,Praga
3,3517,1995,93,7,3,Ochota
4,3013,1992,144,6,5,Mokotow


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   m2.price           1000 non-null   int64 
 1   construction.year  1000 non-null   int64 
 2   surface            1000 non-null   int64 
 3   floor              1000 non-null   int64 
 4   no.rooms           1000 non-null   int64 
 5   district           1000 non-null   object
dtypes: int64(5), object(1)
memory usage: 47.0+ KB


In [5]:
X = df.drop(columns=['m2.price'])
y = df['m2.price']

In [6]:
X['district'].unique()

array(['Srodmiescie', 'Bielany', 'Praga', 'Ochota', 'Mokotow', 'Ursus',
       'Zoliborz', 'Wola', 'Bemowo', 'Ursynow'], dtype=object)

In [7]:
import category_encoders as ce

X = ce.OneHotEncoder(cols=['district']).fit_transform(X, y)


c:\users\marys\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [8]:
X.head()

,construction.year,surface,floor,no.rooms,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10
0,1953,25,3,1,1,0,0,0,0,0,0,0,0,0
1,1992,143,9,5,0,1,0,0,0,0,0,0,0,0
2,1937,56,1,2,0,0,1,0,0,0,0,0,0,0
3,1995,93,7,3,0,0,0,1,0,0,0,0,0,0
4,1992,144,6,5,0,0,0,0,1,0,0,0,0,0


In [9]:
def print_results(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f'RMSE: {round(rmse, 4)}')
    print(f'R squared: {round(r2_score(y_true, y_pred), 4)}')

In [4]:
from sklearn.metrics import f1_score, accuracy_score
def print_results_clf(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    print(f'F1 score: {round(f1, 4)}')
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {round(accuracy, 4)}')

# Model bez normalizacji

In [3]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [12]:
svr = svm.SVR()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8) 

In [14]:
svr.fit(X_train, y_train)

SVR()

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
y_pred = svr.predict(X_test)
print_results(y_test, y_pred)

RMSE: 974.3631
R squared: -0.0127


# Normalizacja
Znormalizujemy kolumny `contruction.year`, `surface`, `floor` oraz `no.rooms`.

### MinMaxScaler

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [18]:
pipe = make_pipeline(MinMaxScaler(), svm.SVR())
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print_results(y_test, y_pred)

RMSE: 949.4816
R squared: 0.0383


### StandardScaler

In [19]:
pipe = make_pipeline(StandardScaler(), svm.SVR())
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print_results(y_test, y_pred)

RMSE: 949.891
R squared: 0.0375


Widzimy, że normalizacja danych rzeczywiście wpływa korzystnie na efektywność modelu. Jednak nie jest to duża różnica.
## Random Search
Początkowo przeprowadzimy random search dla kernelu `rbf`.
Następnie sprawdzimy ten model dla innych kerneli.

In [1]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [21]:
params = {
    'gamma': [0.01, 0.1, 1, 10],
    'C': [1, 10, 100, 1000],
    'kernel': ['rbf']
}
random_search = RandomizedSearchCV(svm.SVR(), params, n_iter=10)
pipe = make_pipeline(MinMaxScaler(), 
                     random_search)
search = random_search.fit(X_train, y_train)

In [22]:
search.best_params_

{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}

In [23]:
pipe = make_pipeline(MinMaxScaler(), svm.SVR(kernel='rbf',
                                            gamma=0.01,
                                            C=1000))
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [24]:
print_results(y_test, y_pred)

RMSE: 342.9967
R squared: 0.8745


Teraz sprawdzimy czy model z kernelem wielomianowym radzi sobie lepiej.

In [25]:
params = {
    'gamma': ['scale'],
    'degree': [ 3, 4, 6, 7, 8, 10, 12, 15, 20, 25, 30],
    'kernel': ['poly']
}
random_search = GridSearchCV(svm.SVR(), params, scoring='r2', cv=5)
pipe = make_pipeline(MinMaxScaler(), 
                     random_search)
search = random_search.fit(X_train, y_train)

In [26]:
search.best_params_

{'degree': 30, 'gamma': 'scale', 'kernel': 'poly'}

In [27]:
search.best_estimator_

SVR(degree=30, kernel='poly')

In [28]:
pipe = make_pipeline(MinMaxScaler(), svm.SVR(kernel='poly',
                                            degree=30, gamma='scale'))
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print_results(y_test, y_pred)

RMSE: 132553.4727
R squared: -18741.9333


## Wnioski
Na danych testowych błąd jest ogromny, wnioskuję z tego, że tak wysoki stopień wielomianu powoduje znaczący overfitting.
Z racji tego, że danych jest mało spróbuję jeszcze raz przeprowadzić strojenie parametrów tym razem na niższych stopniach wielomianu i z krosswalidacją dwukrotną. 

Aby zmniejszyć overfitting spróbuję zwiększyć parametr `C` odpowiadający za regularyzację.

In [55]:
params = {
    'gamma': ['scale'],
    'degree': [ 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'kernel': ['poly'],
    'C': [1, 10, 100, 1000]
}
random_search = RandomizedSearchCV(svm.SVR(), params, scoring='r2', cv=2)
pipe = make_pipeline(MinMaxScaler(), 
                     random_search)
search = random_search.fit(X_train, y_train)

In [56]:
search.best_params_

{'kernel': 'poly', 'gamma': 'scale', 'degree': 3, 'C': 1000}

In [57]:
pipe = make_pipeline(MinMaxScaler(), svm.SVR(kernel='poly',
                                            degree=3, gamma='scale', C=1000))
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print_results(y_test, y_pred)

RMSE: 165.1161
R squared: 0.9709


## Podsumowanie
Wynik przewyższył moje oczekiwania. Metryka $R^2$ jest bardzo bliska 1 co oznacza, że model jest dobry.

Porównując SVR z innymi modelami widzę, że strojenie parametrów w przypadku tego modelu jest kluczowe. Model z niedobranymi parametrami do niczego się nie nadawał.

**Wnioski**
1. W trakcie uczenia modelu dane należy znormalizować. W tym zadaniu minimalnie lepiej poradził sobie MinMaxScaler. 
2. Należy unikać zbyt wysokich stopni wielomianu, gdyż prowadzi to do overfittingu.
3. Warto dostosowywać pod siebie parametr C odpowiadający za regularyzację.
4. Dobrym pomysłem jest przetestowanie różnych kerneli.

# Model dla zbioru heart
Poniżej skorzystamy z SVM dla problemu klasyfikacji.
Opis danych:
- age
- sex
- chest pain type (4 values)
- resting blood pressure
- serum cholestoral in mg/dl
- fasting blood sugar > 120 mg/dl
- resting electrocardiographic results (values 0,1,2)
- maximum heart rate achieved
- exercise induced angina
- oldpeak = ST depression induced by exercise relative to rest
- the slope of the peak exercise ST segment
- number of major vessels (0-3) colored by flourosopy
- thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

In [7]:
heart = pd.read_csv('heart.csv')
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [9]:
X_ = heart.drop(columns=['target'])
y_ = heart.target

In [10]:
y_.unique()

array([1, 0], dtype=int64)

Mamy do czynienia z klasyfikacja binarną.

In [11]:
from sklearn.model_selection import train_test_split

X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_, test_size=0.1, train_size=0.9) 

## Bez normalizacji

In [12]:
svc = svm.SVC()
svc.fit(X_train_, y_train_)
y_pred = svc.predict(X_test_)
print_results_clf(y_test_, y_pred)

F1 score: 0.7179
Accuracy: 0.6452


## Po normalizacji

In [13]:
pipe = make_pipeline(MinMaxScaler(), svm.SVC())
pipe.fit(X_train_, y_train_)
y_pred = pipe.predict(X_test_)
print_results_clf(y_test_, y_pred)

F1 score: 0.8387
Accuracy: 0.8387


In [14]:
pipe = make_pipeline(StandardScaler(), svm.SVC())
pipe.fit(X_train_, y_train_)
y_pred = pipe.predict(X_test_)
print_results_clf(y_test_, y_pred)

F1 score: 0.8
Accuracy: 0.8065


Ponownie normalizacja okazała się  korzystna. Tym razem miała większe znaczenie.  Lepiej sprawdziła się normalizacja przy pomocy MinMaxScaler.

## Random Search i Grid Search

In [43]:
params = {
    'kernel': ['rbf'],
    'gamma': [0.01, 0.02, 0.05, 0.1, 1, 2, 5, 'scale', 'auto'] ,
    'C': [0.1, 1, 10, 100, 1000]
}

rand_search = RandomizedSearchCV(svm.SVC(), params, cv=3, n_iter=20, scoring='accuracy')
pipe = make_pipeline(MinMaxScaler(), rand_search)
search = rand_search.fit(X_train_, y_train_)

In [44]:
search.best_params_

{'kernel': 'rbf', 'gamma': 0.05, 'C': 10}

In [46]:
pipe = make_pipeline(MinMaxScaler(), svm.SVC(kernel='rbf', gamma=0.05, C=10))
pipe.fit(X_train_, y_train_)
y_pred = pipe.predict(X_test_)
print_results_clf(y_test_, y_pred)

F1 score: 0.8125
Accuracy: 0.8065


Wynik trochę się pogorszył.

In [47]:
params = {
    'kernel': ['poly'],
    'gamma': ['scale'],
    'degree': [2, 3, 4, 5, 6, 7, 8, 9],
    'C': [1, 10, 100, 1000]
}

rand_search = RandomizedSearchCV(svm.SVC(), params, cv=2, scoring='accuracy')
pipe = make_pipeline(MinMaxScaler(), rand_search)
search = rand_search.fit(X_train_, y_train_)

In [48]:
search.best_params_

{'kernel': 'poly', 'gamma': 'scale', 'degree': 4, 'C': 1000}

In [49]:
pipe = make_pipeline(MinMaxScaler(), svm.SVC(gamma='scale', degree=4, C=1000, kernel='poly'))
pipe.fit(X_train_, y_train_)
y_pred = pipe.predict(X_test_)
print_results_clf(y_test_, y_pred)

F1 score: 0.8
Accuracy: 0.8065


## Podsumowanie
Wyniki dla kernelu `poly` wyszły gorsze niż dla kernelu `rbf`. Ogólnie sam proces strojenia parametrów nie poprawił działania modelu.

**Wnioski**
1. Ważna jest normalizacja danych.
2. Czasem wyniku nie da się poprawić zmieniając tylko parametry.